In [17]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [18]:
ex1 = pd.read_parquet('train/4.청구입금정보/2018_청구정보_전체.parquet')

In [19]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '

In [20]:
cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']
print(ex1[cols].head())

  대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법 할인건수_R3M 할인건수_B0M
0     자동이체           미확인    당사페이앱+이메일   문자메세지    1회 이상    1회 이상
1     자동이체           주거지           우편      우편    1회 이상    1회 이상
2     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
3     자동이체           주거지           우편      우편    1회 이상    1회 이상
4     자동이체           주거지           우편      우편    1회 이상    1회 이상


In [21]:
print("할인건수_R3M unique values:")
print(ex1['할인건수_R3M'].unique())

print("\n할인건수_B0M unique values:")
print(ex1['할인건수_B0M'].unique())

할인건수_R3M unique values:
['1회 이상' '10회 이상' '20회 이상' '30회 이상' '40회 이상']

할인건수_B0M unique values:
['1회 이상' '10회 이상']


In [22]:
count_cols2 = ['할인건수_R3M', '할인건수_B0M']

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex1[col] = ex1[col].astype(str).str.replace('회 이상', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [23]:
le_cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [24]:
ex1.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,0,0,2,2,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,0,1,5,4,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,0,0,6,5,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,0,1,5,4,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,0,1,5,4,0,1,1,...,0,1,1,0,0,0,0,0,0,0


In [25]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_

In [26]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.001 ].index.tolist()
low_var

['대표결제방법코드']

In [27]:
missing_mask = ex1.isna() | (ex1 == -1)

missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.3].index.tolist()

print("결측(또는 -1) 비율 > 30% 컬럼:", high_na)

결측(또는 -1) 비율 > 30% 컬럼: []


In [28]:
print(ex1['대표결제일'].unique())

[27 13  1  5 20 25 23 15 12 21 10]


In [29]:
cols_to_drop = ['대표결제방법코드']
ex1.drop(columns=cols_to_drop, inplace=True)

In [30]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: ['대표청구서수령지구분코드', '청구서수령방법', '청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature         VIF
2       대표청구서수령지구분코드  118.889631
3            청구서수령방법  118.808725
7            청구금액_B0   11.609959
8           청구금액_R3M   19.879950
10   포인트_마일리지_건별_B0M   81.410345
11   포인트_마일리지_건별_R3M  110.774195
12    포인트_포인트_건별_B0M   30.510475
13    포인트_포인트_건별_R3M   34.298948
14  포인트_마일리지_월적립_B0M   30.002055
15  포인트_마일리지_월적립_R3M   36.844567
19     포인트_적립포인트_R3M   10.457670
21     포인트_이용포인트_R3M   16.059861
23     마일_적립포인트_R12M   15.916083
24      마일_적립포인트_R3M   57.292190
25     마일_이용포인트_R12M   40.487618
26      마일_이용포인트_R3M   40.905447
29          할인금액_R3M   20.

In [31]:
cols_to_drop = ['대표청구서수령지구분코드']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature         VIF
6            청구금액_B0   11.609919
7           청구금액_R3M   19.879933
9    포인트_마일리지_건별_B0M   81.410126
10   포인트_마일리지_건별_R3M  110.774195
11    포인트_포인트_건별_B0M   30.510473
12    포인트_포인트_건별_R3M   34.298876
13  포인트_마일리지_월적립_B0M   30.001736
14  포인트_마일리지_월적립_R3M   36.844497
18     포인트_적립포인트_R3M   10.457124
20     포인트_이용포인트_R3M   16.058813
22     마일_적립포인트_R12M   15.915891
23      마일_적립포인트_R3M   57.291851
24     마일_이용포인트_R12M   40.487477
25      마일_이용포인트_R3M   40.905131
28          할인금액_R3M   20.882071
30          할인금액_B0M   18.204200
31      할인금액_청구서_R3M   15.812532
32      할인금액_청구서_B0M

In [32]:
cols_to_drop = ['포인트_마일리지_건별_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature        VIF
6            청구금액_B0  11.597770
7           청구금액_R3M  19.866276
9    포인트_마일리지_건별_B0M  48.025778
10    포인트_포인트_건별_B0M  30.510434
11    포인트_포인트_건별_R3M  34.298802
12  포인트_마일리지_월적립_B0M  29.712965
13  포인트_마일리지_월적립_R3M  36.065602
17     포인트_적립포인트_R3M  10.457116
19     포인트_이용포인트_R3M  16.058586
21     마일_적립포인트_R12M  15.915081
22      마일_적립포인트_R3M  52.946721
23     마일_이용포인트_R12M  40.197674
24      마일_이용포인트_R3M  40.008372
27          할인금액_R3M  20.882048
29          할인금액_B0M  18.200535
30      할인금액_청구서_R3M  15.812362
31      할인금액_청구서_B0M  14.873460
32     상환개월수_결제일_R6M  11.731907
33     상환개월수_결제일_R3M  14.

In [33]:
cols_to_drop = ['마일_적립포인트_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature        VIF
6            청구금액_B0  11.586211
7           청구금액_R3M  19.865441
9    포인트_마일리지_건별_B0M  35.618123
10    포인트_포인트_건별_B0M  30.510381
11    포인트_포인트_건별_R3M  34.298604
12  포인트_마일리지_월적립_B0M  29.699734
13  포인트_마일리지_월적립_R3M  29.244151
17     포인트_적립포인트_R3M  10.457114
19     포인트_이용포인트_R3M  16.057041
21     마일_적립포인트_R12M  14.518764
22     마일_이용포인트_R12M  39.740472
23      마일_이용포인트_R3M  39.809208
26          할인금액_R3M  20.876405
28          할인금액_B0M  18.196562
29      할인금액_청구서_R3M  15.812137
30      할인금액_청구서_B0M  14.871951
31     상환개월수_결제일_R6M  11.731863
32     상환개월수_결제일_R3M  14.522942
37        혜택수혜금액_R3M  14.212302
38   포인트_

In [35]:
cols_to_drop = ['마일_이용포인트_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_이용포인트_R12M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature        VIF
6            청구금액_B0  11.581140
7           청구금액_R3M  19.865203
9    포인트_마일리지_건별_B0M  33.445322
10    포인트_포인트_건별_B0M  30.510364
11    포인트_포인트_건별_R3M  34.298604
12  포인트_마일리지_월적립_B0M  29.365917
13  포인트_마일리지_월적립_R3M  29.243907
17     포인트_적립포인트_R3M  10.457114
19     포인트_이용포인트_R3M  16.056854
21     마일_적립포인트_R12M  14.040945
22     마일_이용포인트_R12M  28.876646
25          할인금액_R3M  20.875645
27          할인금액_B0M  18.196527
28      할인금액_청구서_R3M  15.812061
29      할인금액_청구서_B0M  14.871942
30     상환개월수_결제일_R6M  11.731815
31     상환개월수_결제일_R3M  14.522824
36        혜택수혜금액_R3M  14.212277
37   포인트_마일리지_환산_B0M  25.494796
38            혜택수혜금액  14.

In [36]:
cols_to_drop = ['포인트_포인트_건별_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_건별_B0M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_이용포인트_R12M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature        VIF
6            청구금액_B0  11.578969
7           청구금액_R3M  19.865198
9    포인트_마일리지_건별_B0M  33.444690
11  포인트_마일리지_월적립_B0M  29.364960
12  포인트_마일리지_월적립_R3M  29.243218
16     포인트_적립포인트_R3M  10.287346
18     포인트_이용포인트_R3M  15.976925
20     마일_적립포인트_R12M  14.040543
21     마일_이용포인트_R12M  28.876581
24          할인금액_R3M  20.852944
26          할인금액_B0M  18.194141
27      할인금액_청구서_R3M  15.811881
28      할인금액_청구서_B0M  14.871896
29     상환개월수_결제일_R6M  11.729915
30     상환개월수_결제일_R3M  14.518225
35        혜택수혜금액_R3M  14.201854
36   포인트_마일리지_환산_B0M  25.490125
37            혜택수혜금액  14.285410


In [37]:
cols_to_drop = ['포인트_마일리지_건별_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['청구금액_B0', '청구금액_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_이용포인트_R12M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']
             feature        VIF
6            청구금액_B0  11.576902
7           청구금액_R3M  19.863681
10  포인트_마일리지_월적립_B0M  28.017009
11  포인트_마일리지_월적립_R3M  29.239348
15     포인트_적립포인트_R3M  10.287050
17     포인트_이용포인트_R3M  15.976332
19     마일_적립포인트_R12M  13.849826
20     마일_이용포인트_R12M  20.012115
23          할인금액_R3M  20.844204
25          할인금액_B0M  18.192300
26      할인금액_청구서_R3M  15.811415
27      할인금액_청구서_B0M  14.871896
28     상환개월수_결제일_R6M  11.729712
29     상환개월수_결제일_R3M  14.518221
34        혜택수혜금액_R3M  14.187499
35   포인트_마일리지_환산_B0M  18.724976
36            혜택수혜금액  14.280334


In [38]:
cols_to_drop = ['청구금액_R3M','포인트_마일리지_월적립_R3M','포인트_이용포인트_R3M','마일_이용포인트_R12M','할인금액_R3M','상환개월수_결제일_R3M','혜택수혜금액']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['할인금액_청구서_R3M', '할인금액_청구서_B0M', '포인트_마일리지_환산_B0M']
            feature        VIF
21     할인금액_청구서_R3M  15.333832
22     할인금액_청구서_B0M  14.825316
29  포인트_마일리지_환산_B0M  11.769333


In [39]:
cols_to_drop = ['할인금액_청구서_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

VIF > 10인 컬럼들: ['포인트_마일리지_환산_B0M']
            feature        VIF
28  포인트_마일리지_환산_B0M  11.766797


In [40]:
cols_to_drop = ['포인트_마일리지_환산_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [41]:
ex1.to_parquet('청구입금정보_전처리.parquet', index=False)